**Loading Dataset & Data Augmentation**

In [7]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_gen = datagen.flow_from_directory(
    "/content/drive/MyDrive/Emtech2/Fruits",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_gen = datagen.flow_from_directory(
    "/content/drive/MyDrive/Emtech2/Fruits",
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

# Check class count and image shape
print("Train classes:", train_gen.num_classes)
print("Train shape:", train_gen.image_shape)



Found 359 images belonging to 9 classes.
Found 359 images belonging to 9 classes.
Train classes: 9
Train shape: (128, 128, 3)


**Creating CNN Model For Fruit Classification**

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     4,194,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │         2,313 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,587,209 (17.50 MB)

 Trainable params: 4,586,249 (17.50 MB)

 Non-trainable params: 960 (3.75 KB)

In [9]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.2, verbose=1)


**Training Model**

In [10]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[checkpoint, early, reduce_lr]
)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2292 - loss: 8.7495

12/12 ━━━━━━━━━━━━━━━━━━━━ 63s 5s/step - accuracy: 0.2341 - loss: 8.7139 - val_accuracy: 0.1532 - val_loss: 2.2597 - learning_rate: 0.0010
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3873 - loss: 5.8065

12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 0.3877 - loss: 5.7799 - val_accuracy: 0.1783 - val_loss: 3.5879 - learning_rate: 0.0010
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 53s 5s/step - accuracy: 0.3695 - loss: 4.5463 - val_accuracy: 0.1560 - val_loss: 4.7714 - learning_rate: 0.0010
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3657 - loss: 3.3417
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
12/12 ━━━━━━━━━━━━━━━━━━━━ 56s 5s/step - accuracy: 0.3682 - loss: 3.3506 - val_accuracy: 0.1476 - val_loss: 4.4675 - learning_rate: 0.0010
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.4013 - loss: 2.4918 - val_accuracy: 0.1114 - val_loss: 4.7992 - learning_rate: 2.0000e-04
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 81s 4s/step - accuracy: 0.4414 - loss: 2.1115 - val_accuracy: 0.1142 - val_loss: 5.4804 - learning_rate: 2.0000e-04
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4126 - loss: 2.1750
Epoch 7: ReduceLROnPl

In [6]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30  # Increase for better learning
)


Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.4052 - loss: 1.5329 - val_accuracy: 0.1866 - val_loss: 9.3952
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.4267 - loss: 1.6047 - val_accuracy: 0.2256 - val_loss: 8.5231
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.4169 - loss: 1.6018 - val_accuracy: 0.2786 - val_loss: 7.4968
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.4276 - loss: 1.4511 - val_accuracy: 0.3092 - val_loss: 5.9791
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.4867 - loss: 1.5640 - val_accuracy: 0.2786 - val_loss: 6.5466
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - accuracy: 0.3444 - loss: 1.6803 - val_accuracy: 0.4540 - val_loss: 2.5297
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.3542 - loss: 1.6106 - val_accuracy: 0.3900 - val_loss: 2.6718
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.4658 - loss: 1.4501 - val_accuracy: 0.4903 - val_loss:

**Saving Model**

In [11]:
model.save('fruits_model.keras')